<a href="https://colab.research.google.com/github/InnaAndreeva/3_dz/blob/master/3lab_final1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf 
from tensorflow.python.keras.datasets import cifar10  
import numpy as np 
import time
from keras.utils import np_utils
from tensorflow.python import debug as tf_debug

Using TensorFlow backend.


In [2]:
tf.__version__

'1.15.0'

In [3]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
nb_classes = 10
print(y_train.shape)
y_train = y_train.reshape(y_train.shape[0])  # somehow y_train comes as a 2D nx1 matrix
y_test = y_test.reshape(y_test.shape[0])
x_train = X_train.astype('float32')
x_test = X_test.astype('float32')
x_train /= 255
x_test /= 255
x_train = np.reshape(x_train, [-1, 3072])
x_test  = np.reshape(x_test, [-1, 3072])
y_train = np_utils.to_categorical(y_train, nb_classes)
y_test = np_utils.to_categorical(y_test, nb_classes)
import os

170500096/170498071 [==============================] - 2s 0us/step
(50000, 1)


In [0]:
def weight_variable(shape): # создание переменной весов слоя
    initial = tf.truncated_normal(shape, stddev=0.1) 
    return tf.Variable(initial)

def bias_variable(shape): 
    initial = tf.constant(0.1, shape=shape) 
    return tf.Variable(initial)

def conv2d(x_, W):  # операция свертки
    return tf.nn.conv2d(x_, W, strides=[1, 1, 1, 1], padding='SAME')

In [0]:
def max_pool_2x2(x_): # операция пулинга 
    return tf.nn.max_pool(x_, ksize=[1, 2, 2, 1], 
                          strides=[1, 2, 2, 1], padding='SAME')
def conv_layer(input, shape): # сверточный слой
    W = weight_variable(shape) # переменная для весов свертки
    b = bias_variable([shape[3]]) # свободные члены = количество фильтров
    return tf.nn.relu(conv2d(input, W) + b)
def full_layer(input, size): 
    in_size = int(input.get_shape()[1]) 
    W = weight_variable([in_size, size]) 
    b = bias_variable([size]) 
    return tf.matmul(input, W) + b

In [0]:
x = tf.placeholder(tf.float32, shape=[None, 3072])  # создаем переменную для x
y_ = tf.placeholder(tf.float32, shape=[None, 10]) 
keep_prob = tf.placeholder(tf.float32)  # переменная для dropout
x_image = tf.reshape(x, [-1, 32, 32, 3])  # преобразование в размеры картинки

In [7]:
with tf.name_scope('conv_1'): 
    conv1 = conv_layer(x_image, shape=[3, 3, 3, 32]) # создание сверточного слоя    
    conv1_pool = max_pool_2x2(conv1) # слой пулинга
with tf.name_scope('conv_2'): 
    conv2 = conv_layer(conv1_pool, shape=[3, 3, 32, 64]) 
    conv2_pool = max_pool_2x2(conv2) 
with tf.name_scope('conv_3'): 
#     conv3 = conv_layer(conv2, shape=[3, 3, 64, 128]) 
    conv3 = conv_layer(conv2_pool, shape=[3, 3, 64, 128]) 
    conv3_pool = max_pool_2x2(conv3) 
    conv3_flat = tf.reshape(conv3_pool, [-1, 4*4*128]) 
with tf.name_scope('full_1'): 
    full_1 = tf.nn.relu(full_layer(conv3_flat, 512)) 
with tf.name_scope('dropout'): 
    full1_drop = tf.nn.dropout(full_1, keep_prob=keep_prob) 
with tf.name_scope('activations'): 
    y_conv = full_layer(full1_drop, 10) 
    tf.summary.scalar('cross_entropy_loss',y_conv)
with tf.name_scope('cross'): 
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=y_conv, labels=y_)) 
# #SGD 
train_step = tf.train.AdamOptimizer(learning_rate=0.0005).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1)) 
accuracy = tf.reduce_mean(tf.cast(correct_prediction , tf.float32))


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [8]:
with tf.Session() as sess: 
    sess.run(tf.global_variables_initializer()) 
    start_time = time.time()
    for epochs in range(5):
        for i in range(2500):  
            batch = [x_train[20*i:20*(i+1)] , y_train[20*i:20*(i+1)]]

            if i % 250 == 0:       
                train_accuracy = sess.run(accuracy, feed_dict={x: batch[0], 
                                                               y_: batch[1], keep_prob: 1.0}) 
                print("time {}, step {}, training accuracy {}".format(time.time() - start_time,i, train_accuracy)) 
            sess.run(train_step, feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
        print('-----------------------------------------------Эпоха№', epochs)
        X = x_test.reshape(10, 1000, 3072) 
        Y = y_test.reshape(10, 1000, 10) 
        test_accuracy = np.mean([sess.run(accuracy, feed_dict={x:X[i], y_:Y[i], keep_prob:1.0}) for i in range(10)])
        print("test accuracy: {}".format(test_accuracy))


time 0.1654977798461914, step 0, training accuracy 0.10000000149011612
time 12.334223985671997, step 250, training accuracy 0.20000000298023224
time 24.44785714149475, step 500, training accuracy 0.550000011920929
time 36.69976234436035, step 750, training accuracy 0.44999998807907104
time 48.92852592468262, step 1000, training accuracy 0.44999998807907104
time 61.22667956352234, step 1250, training accuracy 0.5
time 73.58510637283325, step 1500, training accuracy 0.5
time 85.92834329605103, step 1750, training accuracy 0.6000000238418579
time 98.25592517852783, step 2000, training accuracy 0.44999998807907104
time 110.75135207176208, step 2250, training accuracy 0.800000011920929
-----------------------------------------------Эпоха№ 0
test accuracy: 0.5436999201774597
time 129.17269706726074, step 0, training accuracy 0.6499999761581421
time 141.76247787475586, step 250, training accuracy 0.550000011920929
time 154.28582215309143, step 500, training accuracy 0.75
time 166.818858623504